In [270]:
include("StackyFan.jl")

coneRayDecomposition (generic function with 1 method)

In [293]:
"""

    BerghA(F::StackyFan,D::Array{Int64,1})

    Given a stacky fan F and a vector of booleans D representing the distinguished structure,
    returns a smooth stacky fan where the distinguished rays are independent.

    
"""

function BerghA(F::StackyFan,D::Array{Int64,1})
    X=deepcopy(F)
    rayMatrix=convert(Array{Int64,2},Array(Polymake.common.primitive(X.fan.RAYS)))
    coneList=getCones(X.fan)
    dim=size(rayMatrix,2)
    numRays=size(rayMatrix,1)
    stackTracker=ones(Int64,numRays)
    
    #check if the vector D has length equal to the number of rays in F
    if numRays != size(D,1)
        error("length of vector representing distinguished structure does not agree with number of rays in stacky fan.")
    end
    
    #A0: initialization
    while(true)
        rayMatrix=convert(Array{Int64,2},Array(Polymake.common.primitive(X.fan.RAYS)))
        println(rayMatrix)
        coneList=getCones(X.fan)
        println(coneList)
        println(stackyWeights(X))
        println()
        #A1: check if finished
        # Find S the set of cones that contain a distinguised ray and an interior a lattice point 
        #Note: cones in S are 1-indexed.
        S=filter(cone->distinguishedAndIntPoint(cone,rayMatrix,D),coneList)
        # If S is empty, the program terminates.
        if S==[]
            break
        end
        
        #A2 - find extremal cones
        Smax=extremalCones(S,rayMatrix,D)
        
        #A2 - find interior points in Smax
        intPoints=[]
        for cone in Smax
            #C=rowMinors(rayMatrix,cone)
            C=coneConvert(cone,rayMatrix)
            coneIntPoints=interiorPoints(C)
            if coneIntPoints==nothing
                # does the program ever reach this line if it breaks at S empty?
                return C
            end
            for point in coneIntPoints
               push!(intPoints,(point,cone)) 
            end
        end
        
        #A2 - find stacky points (in terms of coefficients) derived from interior points
        P=Array{Int64,1}[]
        for (point,cone) in intPoints
            stackyPoint=coneRayDecomposition(cone,rayMatrix,point,stackyWeights(X))
            push!(P,stackyPoint)
        end
        
        #A2 - find smallest element of P with respect to lex ordering.
        psi=minimalByLex(P)
        
        #A3 - perform root construction
        X=rootConstructionDistinguishedIndices(X,D,psi)
        
        #A3 - modify psi with respect to root construction
        for i in 1:length(psi)
            if D[i]==1 && psi[i]>0
                psi[i]=1
            end
        end
        
        #TODO: Currently the distinguished rays D and psi coefficients are not
        #used in the output. Should these be removed? Also may need to annotate
        #what stackTracker does.
        
        # Convert arrays to dictionaries as constructing fan objects through subdivision
        # may reorder rays
        code_rays = mapslices(encode, Polymake.common.primitive(X.fan.RAYS), dims=2)
        # Track the indices of distinguished rays
        D_pairs = map((x,y) -> (x,y), code_rays, D)
        D_Dict = Dict(D_pairs)
        # Track psi as a linear combination of the generators
        psiPairs = map((x,y) -> (x,y), code_rays,psi)
        psiDict = Dict(psiPairs)
        # 
        trackerPairs = map((x,y) -> (x,y), code_rays, stackTracker)
        trackerDict= Dict(trackerPairs)
        
        #A5 - repeat star subdivision
        while(count(x->x>0,psi)>1)
            #A4 - perform stacky star subdivision
            # Get the indices of the non-zero coefficients in psi
            supportCone=findall(x->x!=0,psi)
            exceptional=findBarycenter(supportCone,X.fan)
            
            X=stackyBlowup(X,[x-1 for x in supportCone],exceptional)
            
            G=gcd(exceptional)
            primExcep=Polymake.common.primitive(exceptional)
            
            # Update the dictionaries storing fan information
            D_Dict[encode(primExcep)]=1
            psiDict[encode(primExcep)]=1
            trackerDict[encode(primExcep)]=G
        
            #A4 - modify psi
            newpsi=zeros(Int64,length(psi))
            for i in 1:length(psi)
                if psi[i]>1
                    newpsi[i]=psi[i]-1
                end
            end
            psi=newpsi
        end
        
        # Convert dictionaries back into arrays in the order of the new rays
        newRays=slicematrix(convert(Array{Int64,2},Array(Polymake.common.primitive(X.fan.RAYS))))
        newD=Int64[]
        newpsi=Int64[]
        newTracker=Int64[]
        for ray in newRays
            E=encode(ray)
            push!(newD,D_Dict[E])
            push!(newpsi,psiDict[E])
            push!(newTracker,trackerDict[E])
        end
        psi=newpsi
        D=newD
        stackTracker=newTracker
    end
    
    return X, stackTracker
end

BerghA (generic function with 1 method)

In [294]:
X=Polymake.fulton.NormalToricVariety(INPUT_RAYS=[1 2 3;1 5 4; 6 11 8],INPUT_CONES=[[0,1,2]])

type: NormalToricVariety

INPUT_CONES
	{0 1 2}

INPUT_RAYS
	1 2 3
	1 5 4
	6 11 8

In [295]:
F=addStackStructure(X,[3,4,2])

StackyFan(Polymake.BigObjectAllocated(Ptr{Nothing} @0x000000001c5418c0), [3, 4, 2], Dict("1,5,4" => 4, "1,2,3" => 3, "6,11,8" => 2))

In [296]:
Y=BerghA(F,[1,1,1])

[1 2 3; 1 5 4; 6 11 8]
[[1, 2, 3], [1, 2], [1, 3], [2, 3], [1], [2], [3]]
 
[1 2 3; 1 5 4; 8 18 15; 6 11 8]
[[1, 2, 3], [1, 3, 4], [2, 3, 4], [1, 2], [1, 3], [2, 3], [1, 4], [3, 4], [2, 4], [1], [2], [3], [4]]
 
[1 2 3; 1 5 4; 8 18 15; 6 11 8; 15 34 27]
[[1, 2, 3], [1, 3, 4], [2, 3, 5], [3, 4, 5], [2, 4, 5], [1, 2], [1, 3], [2, 3], [1, 4], [3, 4], [3, 5], [2, 5], [4, 5], [2, 4], [1], [2], [3], [4], [5]]
 
[1 2 3; 1 5 4; 8 18 15; 6 11 8; 15 34 27; 29 63 50]
[[1, 2, 3], [1, 3, 4], [2, 3, 5], [2, 4, 5], [3, 4, 6], [3, 5, 6], [4, 5, 6], [1, 2], [1, 3], [2, 3], [1, 4], [3, 4], [3, 5], [2, 5], [2, 4], [4, 5], [4, 6], [3, 6], [5, 6], [1], [2], [3], [4], [5], [6]]
 
[1 2 3; 1 5 4; 8 18 15; 6 11 8; 15 34 27; 29 63 50; 50 108 85]
[[1, 2, 3], [1, 3, 4], [2, 3, 5], [2, 4, 5], [3, 4, 6], [3, 5, 6], [4, 5, 7], [4, 6, 7], [5, 6, 7], [1, 2], [1, 3], [2, 3], [1, 4], [3, 4], [3, 5], [2, 5], [2, 4], [4, 5], [4, 6], [3, 6], [5, 6], [4, 7], [5, 7], [6, 7], [1], [2], [3], [4], [5], [6], [7]]
 
[1 2 3; 1 5 4

┌ Warning: Interrupting polymake is not safe.
│ SIGINT is disabled while waiting for polymake to finish its computations.
└ @ Polymake /home/kitty/.julia/packages/Polymake/6iwMj/src/perlobj.jl:47


LoadError: InterruptException:

In [45]:
X.RAYS*(1/5)

pm::Matrix<double>
0.2 0.4
0.2 0
